In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from warnings import filterwarnings

filterwarnings('ignore')

In [ ]:
app= pd.read_csv(r"D:\Datasets\Credit Card Fraud Data\application_data.csv")
prev_app= pd.read_csv(r"D:\Datasets\Credit Card Fraud Data\application_data.csv")         


In [ ]:
# pd.set_option() allows you to show all the columns present in the large dataset

pd.set_option('display.max_columns', None)  
app.head()

In [ ]:
prev_app.head()

In [ ]:
app.shape

In [ ]:
prev_app.shape

In [ ]:
app.info(verbose=True)

In [ ]:
app.describe()

In [ ]:
app.describe(include='object')

In [ ]:
# total Null Values
total_null=app.isna().sum()
null_per=((total_null/app.shape[0])*100)

null_df= pd.DataFrame({'total_Null_data': total_null, 'Null percentage': null_per})

null_df.sort_values(by='Null percentage',ascending=False).head(20)

In [ ]:
null_more_than_thresh= (null_df[null_df['Null percentage']>35.0])
null_more_than_thresh.sort_values(by='Null percentage',ascending=False)

In [ ]:
#Plotting the graph for Null cols which are higher than threshold
plt.figure(figsize=(10,4))
px.bar(null_more_than_thresh.sort_values(by='Null percentage', ascending=False),title= 'List of Columns & null counts where null values are more than 35%', labels={'"Null Columns','Count of null values'},template= 'plotly_dark')



Theoretically, 25 to 30% is the maximum missing values are allowed, beyond which we might want to drop the variable from analysis. But practically we get variables with ~50% of missing values but still, the customer insists to have it for analyzing. In those cases, we have to treat them accordingly. Here, we will remove columns with null values of more than 35% after observing those columns.

In [ ]:
#checking the total null column count > 35%

len(null_more_than_thresh)

In [ ]:
# drop the total columns which are > 35%
columns_label= null_more_than_thresh.index.values  # Getting names of all the columns which are having null values > 35%

app.drop(columns_label, axis=1, inplace=True) 

In [ ]:
app.shape

In [ ]:
#Recheck the null counts of the app dataframe

def check_null():
    
    totalNA= app.isna().sum()
    perNA= (totalNA/app.shape[0])*100

    null_check= pd.DataFrame({'totalNA':totalNA, 'Per_NA':perNA})
    null_check=null_check.sort_values(by='Per_NA', ascending=False)
    
    cols_with_null_data= null_check[null_check['Per_NA']>0.0000]

    return cols_with_null_data


In [ ]:
check_null()

In [ ]:
app['OCCUPATION_TYPE'].dtype

In [ ]:
def check_null_dtypes():
    
    cat=[]
    num= []
    
    null_cols= check_null()
    null_col_names= null_cols.index.values
    
    for i in null_col_names:
        if(app[i].dtype=='object'):
            cat.append(i)
        else:
            num.append(i)

    print(f'categorical: {len(cat)}\n\n {cat} \n\n numerical: {len(num)}\n\n {num}')
    return cat, num


In [ ]:
cat, num= check_null_dtypes()

In [ ]:
app['NAME_TYPE_SUITE'].value_counts()

In [ ]:
app['OCCUPATION_TYPE'].value_counts()

In [ ]:
check_null()

In [ ]:
cat[0]

In [ ]:
# Fill the categorical NA columns with mode value, lets leave occupation_type column, as it might contains some imp info

app[cat[1]].fillna(app[cat[1]].mode()[0], inplace=True)
    
# Fill the numerical NA columns with median value
for i in num:

    app[i].fillna(app[i].median(), inplace=True)
    
   

In [ ]:
# Check if all the null values are filled or not

check_null()

In [ ]:
check_null_dtypes()

In [ ]:
# Clear the error values present in ["DAYS_BIRTH","DAYS_EMPLOYED","DAYS_REGISTRATION","DAYS_ID_PUBLISH","DAYS_LAST_PHONE_CHANGE"]

app[["DAYS_BIRTH","DAYS_EMPLOYED","DAYS_REGISTRATION","DAYS_ID_PUBLISH","DAYS_LAST_PHONE_CHANGE"]].head()

From above table we can see that all these columns are Days columns and are having days count as -ve number. So, lets clear it up

In [ ]:
# create a list for all the columns which starts with Days word, as these are the only columns which have -ve days error

days_cols= [i for i in app if i.startswith('DAYS_')]
days_cols

In [ ]:
def check_unique_values(days_cols):
    
    for i in days_cols:
        print(app[i].unique())

check_unique_values(days_cols)

In [ ]:
#Convert negative values to the positive one for all the days columns

app[days_cols]= abs(app[days_cols])

In [ ]:
# check if all the cols after taking abso value of the cols 
check_unique_values(days_cols)

In [ ]:
# abs_days_cols

In [ ]:
#Convert columns which has Y/N value to 0/1 for our convinence

app['FLAG_OWN_CAR']= np.where(app['FLAG_OWN_CAR']=='Y', 1, 0)
app['FLAG_OWN_REALTY']= np.where(app['FLAG_OWN_REALTY']=='Y', 1, 0)

In [ ]:
app.head()

In [ ]:
#check the distribution of categorical columns

app.describe(include='O')

In [ ]:
# app['CODE_GENDER'].value_counts()

def cat_and_num():
    categorical_cols=[]
    numerical_cols=[]

    for i in app.columns:
        if app[i].dtype == 'O':
            categorical_cols.append(i)
        else:
            numerical_cols.append(i)
            
    return categorical_cols, numerical_cols

In [ ]:
# check the value counts of all the categorical columns

total_cat_cols, total_num_cols= cat_and_num()

for i in total_cat_cols:
    print('\n\n',app[i].value_counts())
    

#### After seeing the above categorical description we can say that CODE_GENDER and ORGANIZATION_TYPE has NA value as XNA, so lets impute it


In [ ]:
# Compute and impute the CODE_GENDER XNA values with the most frequent value

app['CODE_GENDER']= app['CODE_GENDER'].replace('XNA','F')

In [ ]:
app['CODE_GENDER'].value_counts()

In [ ]:
# app[app['ORGANIZATION_TYPE']=='XNA']

In [ ]:
cat_df=app[total_cat_cols]

In [ ]:
cat_df[cat_df['ORGANIZATION_TYPE']=='XNA'].head(50)

In [ ]:
# Observation says that ORGANIZATION_TYPE is follwing a parallel pattern with NAME_INCOME_TYPE 
# check how much of data in NAME_INCOME_TYPE is having rows as Pensioner

app['NAME_INCOME_TYPE'].value_counts()

In [ ]:
# As we can see that total count of XNA's in ORGANIZATION_TYPE is 55374 
# and total count of Pensioner in NAME_INCOME_TYPE is 55362
# which says that Pensioner's count is almost same to the XNA, So the value is Missing At Random and we can use it to fill ORGANIZATION_TYPE XNA's

In [ ]:
# Analyse the 'OCCUPATION_TYPE' column as it is yet to be imputed for null values
app_copy= cat_df

In [ ]:
#Replace null values with 'XNA' just to know its relation with other columns
app_copy['OCCUPATION_TYPE'].fillna('XNA', inplace=True)

In [ ]:
app_copy['OCCUPATION_TYPE'].value_counts()

In [ ]:
# Notice that NAME_INCOME_TYPE and OCCUPATION_TYPE is having a pattern together, its that almost 80% of missing data is 
#having 'Pensioner' as most frequent one.

app_copy.loc[(app_copy['NAME_INCOME_TYPE']=='Pensioner') & (app_copy['OCCUPATION_TYPE']=='XNA')]

In [ ]:
# Replace XNA values in ORGANIZATION_TYPE as Pensioner

app['ORGANIZATION_TYPE']= app['ORGANIZATION_TYPE'].replace('XNA', 'Pensioner')

# Impute missing values in OCCUPATION_TYPE column with Pensioner

app['OCCUPATION_TYPE'].fillna('Pensioner', inplace= True)


In [ ]:
cat_df.head(10)

In [ ]:
app['AMT_INCOME_TOTAL'].head(20)

In [ ]:
plt.boxplot(app['AMT_INCOME_TOTAL']);

In [ ]:
# We have some nominal categorical column, so impute them as required

app['AMT_INCOME_TYPE']= pd.qcut(app['AMT_INCOME_TOTAL'], q=[0, 0.2, 0.5, 0.8, 0.95, 1], labels=['VERY_LOW', 'LOW', 'MEDIUM', 'HIGH', 'VERY_HIGH'])
app.head()

In [ ]:
# Lets Bin the DAYS_BIRTh to the AGE_GROUP

app['YEARS_BIRTH']=(app['DAYS_BIRTH']/365).astype('int')

app['YEARS_BIRTH'].unique()

In [ ]:
# Bin the YEARS_BIRTH into AGE_GROUP bins

app['AGE_GROUP']= pd.cut(app['YEARS_BIRTH'], bins=[19, 25, 35, 60, 100], labels=['Very_Young','Young', 'Middle_Age', 'Senior_Citizen'])

In [ ]:
plt.figure(figsize=(22,5))
sns.countplot(data= app, x= 'YEARS_BIRTH')

In [ ]:
plt.figure(figsize=(22,5))
sns.countplot(data=app,x='AGE_GROUP')

In [ ]:
app.head()

In [ ]:
app.info()

In [ ]:
# categorical_cols

In [ ]:
# Convert all the object datatypes to category dtypes

for i in total_cat_cols:
    app[i]= app[i].astype('category')

In [ ]:
# app['NAME_CONTRACT_TYPE']= app['NAME_CONTRACT_TYPE']. astype('category')

In [ ]:
# Here the thing is to notice that the memory usage is reduced quite nicely
app.info()

In [ ]:
# There are many unwanted columns which should be removed or refactored

unwanted=['FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE',
       'FLAG_PHONE', 'FLAG_EMAIL','REGION_RATING_CLIENT','REGION_RATING_CLIENT_W_CITY','FLAG_EMAIL', 'REGION_RATING_CLIENT',
       'REGION_RATING_CLIENT_W_CITY', 'FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_3','FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5', 'FLAG_DOCUMENT_6',
       'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8', 'FLAG_DOCUMENT_9','FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_12',
       'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15','FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18',
       'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21']

In [ ]:
app.drop(labels=unwanted, axis=1, inplace=True)

### Outlier Analysis

In [ ]:
# To detect the outlier, plot a box plot to analyis the numerical column

num_cols= app.select_dtypes(include='number').columns

fig, axes= plt.subplots(nrows= 7, ncols= 5)
fig.subplots_adjust(left= 0, bottom=0, right=3, top=12, wspace= 0.09, hspace= 0.3)

for ax, column in zip(axes.flatten(), num_cols):
    
    sns.boxplot(data= app[column],orient= 'h', ax=ax).set(xlabel= column)


###  Before starting the analysis lets check the data imbalance

In [ ]:
target0 = app.loc[app['TARGET']==0]
target1 = app.loc[app['TARGET']==1]

In [ ]:
#Imbalance Ratio

round(len(target0)/len(target1),2)

In [ ]:
# Visualisation of Imbalance Data distribution on a pie chart

count0= 0
count1= 0

for i in app['TARGET'].values:
    
    if i == 0:
        count0 +=1
    else:
        count1 +=1

count0 = (count0/len(app['TARGET'])) * 100        
count1 = (count1/len(app['TARGET'])) * 100


x= ['Defaulted Population(Target=1)', 'Non-Defaulted Population(Target=0)']
y= [count1, count0]

explode= (0.1, 0) # only explode the 1st slice

fig1, ax1= plt.subplots()
ax1.pie(y, explode= explode, labels= x,autopct='%1.1f%%', shadow= True, startangle= 110)

ax1.axis('equal')     # Equal aspect ratio ensures that pie is drawn as a circle.

plt.title("Data Imbalance Chart", fontsize= 25)
plt.show()

In [ ]:
plt.figure(figsize=(40,5))
plt.rcParams["axes.labelsize"] = 30
plt.rcParams['axes.titlesize'] = 80                                                           # Plot Configuration 
plt.rcParams['axes.titlepad'] = 30
fig, axes = plt.subplots(nrows=1,ncols=2)                  
sns.set_context('talk')
fig.subplots_adjust(left= 0.09,bottom=1,right=3,top= 12,wspace=0.6,hspace=0.9) 


plt.subplot(121)
plt.xscale('log')                                                                             # For Target0      
sns.countplot(data=target0,y='ORGANIZATION_TYPE',
              order=app['ORGANIZATION_TYPE'].value_counts().index,palette='Set3',hue = 'TARGET')
plt.title("ORGANIZATION_TYPE Vs Target 0")



plt.subplot(122)
plt.xscale('log')                                                                              # For Target1
sns.countplot(data=target1, y='ORGANIZATION_TYPE',
              order=app['ORGANIZATION_TYPE'].value_counts().index,palette='Set1',hue = 'TARGET')
plt.title("ORGANIZATION_TYPE Vs Target 1")

plt.show()

In [ ]:
# To understand it properly lets compare all data columns with the target columns

def comp_plot(col):
    
    plt.figure(figsize=(40, 20))

    plt.rcParams['axes.labelpad'] = 50
    plt.subplot(1,2,1)
    sns.countplot(data= target0, x= col, hue= 'TARGET', palette= 'Set2' )
    plt.xlabel(col, fontsize= 30, fontweight="bold")                                                         #Target 0
    plt.ylabel('Non Payment Difficulties', fontsize= 30, fontweight="bold")
    plt.xticks(rotation=90, fontsize=30)
    plt.yticks(rotation=360, fontsize=30)

    plt.rcParams['axes.labelpad'] = 50
    plt.subplot(1,2,2)
    sns.countplot(data= target1, x= col, hue= 'TARGET', palette= 'Set3' )
    plt.xlabel(col, fontsize= 30, fontweight="bold")                                                         #Target 0
    plt.ylabel('Payment Difficulties', fontsize= 30, fontweight="bold")
    plt.xticks(rotation=90, fontsize=30)
    plt.yticks(rotation=360, fontsize=30)
    
    plt.show()

In [ ]:
# Remove Organization_Type from the categorical list as we have already analysed it

cat_list= list(app.select_dtypes(include='category').columns)
cat_list.remove('ORGANIZATION_TYPE')

cat_list

In [ ]:
for i in cat_list:
    comp_plot(i)

In [ ]:
# Way to calculate percent of Male Defaulters and Non-Defaulters

len(app[(app['CODE_GENDER']=='M') & (app['TARGET']==1)])/len(app[(app['TARGET']==1)])*100  # Non-Defaulter Male count
len(app[(app['CODE_GENDER']=='M') & (app['TARGET']==0)])/len(app[(app['TARGET']==0)])*100  # Defaulter Male count

### Univariate Analysis of Numerical Columns W.R.T Target Variable

In [ ]:
def num_univ(col):
    
    sns.set(style='darkgrid')
    plt.figure(figsize=(40,20))
    
    plt.subplot(1,2,1)
    sns.distplot(target0[col], color="g")
    plt.yscale('linear')
    plt.xlabel(col, fontsize= 30, fontweight= 'bold')
    plt.ylabel('Non Payment Difficulties', fontsize= 30, fontweight= 'bold')
    plt.xticks(rotation=90, fontsize=30)
    plt.yticks(rotation=360, fontsize=30)
    
    plt.subplot(1,2,2)
    sns.distplot(target1[col], color="r")
    plt.yscale('linear')
    plt.xlabel(col, fontsize= 30, fontweight= 'bold')
    plt.ylabel('Payment Difficulties', fontsize= 30, fontweight= 'bold')
    plt.xticks(rotation=90, fontsize=30)
    plt.yticks(rotation=360, fontsize=30)
    
    plt.show();

In [ ]:
app.select_dtypes(exclude='category').columns

In [ ]:
num_univ('AMT_ANNUITY')

In [ ]:
num_univ('AMT_CREDIT')

In [ ]:
num_univ('AMT_GOODS_PRICE')

In [ ]:
num_univ(col='AMT_INCOME_TOTAL')

### Bivariate Analysis : Numerical And Categorical w.r.t to Target Variable

In [ ]:
# Select Num and Cat type columns for analysis

app[["TARGET","AMT_INCOME_TOTAL","NAME_EDUCATION_TYPE","NAME_FAMILY_STATUS"]].head()

### For Target 0

In [ ]:
plt.figure(figsize=(35, 14))
plt.yscale('log')                           #As the values are too big, so better to use log

sns.boxplot(data= target0, x= 'NAME_EDUCATION_TYPE', y= 'AMT_INCOME_TOTAL', hue= 'NAME_FAMILY_STATUS', orient= 'v', palette='Set2')

plt.legend(loc='upper right')
plt.title('Income amount vs Education Status',fontsize=35 )
plt.xlabel("NAME_EDUCATION_TYPE",fontsize= 30, fontweight="bold")
plt.ylabel("AMT_INCOME_TOTAL",fontsize= 30, fontweight="bold")
plt.xticks(rotation=90, fontsize=30)
plt.yticks(rotation=360, fontsize=30)

plt.show()

In [ ]:
plt.figure(figsize=(35, 14))
plt.yscale('log')                           #As the values are too big, so better to use log
# plt.xticks(rotation= 90)

sns.boxplot(data= target0, x= 'NAME_EDUCATION_TYPE', y= 'AMT_CREDIT', hue= 'NAME_FAMILY_STATUS', orient= 'v', palette='Set2')

plt.legend(loc='upper right')
plt.title('Credit vs Education Status',fontsize=35 )
plt.xlabel("NAME_EDUCATION_TYPE",fontsize= 30, fontweight="bold")
plt.ylabel("AMT_CREDIT",fontsize= 30, fontweight="bold")
plt.xticks(rotation=90, fontsize=30)
plt.yticks(rotation=360, fontsize=30)

plt.show()

### For Target 1

In [ ]:
plt.figure(figsize=(30,12)) 
plt.yscale('log')                     #As the values are too large, it is convinient to use log for better analysis
# plt.xticks(rotation = 90)


sns.boxplot(data =target1, x='NAME_EDUCATION_TYPE',y='AMT_INCOME_TOTAL',   #Boxplot w.r.t Data Target 1
            hue ='NAME_FAMILY_STATUS',orient='v',palette='Set2')

plt.legend( loc = 'upper right')                                              #Adjusting legend position
plt.title('Income amount vs Education Status',fontsize= 35)
plt.xlabel("NAME_EDUCATION_TYPE",fontsize= 30, fontweight="bold")
plt.ylabel("AMT_INCOME_TOTAL",fontsize= 30, fontweight="bold")
plt.xticks( fontsize=30)
plt.yticks(rotation=360, fontsize=30)

plt.show()

#### Credit Amnt Vs Name_Education_Type Vs Name_Family_Status

In [ ]:
plt.figure(figsize=(30,12)) 
plt.yscale('log')                     #As the values are too large, it is convinient to use log for better analysis
# plt.xticks(rotation = 90)


sns.boxplot(data =target1, x='NAME_EDUCATION_TYPE',y='AMT_CREDIT',   #Boxplot w.r.t Data Target 1
            hue ='NAME_FAMILY_STATUS',orient='v',palette='Set2')

plt.legend( loc = 'upper right')                                              #Adjusting legend position
plt.title('Credit Amount vs Education Status',fontsize= 35)
plt.xlabel("NAME_EDUCATION_TYPE",fontsize= 30, fontweight="bold")
plt.ylabel("AMT_CREDIT",fontsize= 30, fontweight="bold")
plt.xticks( fontsize=30)
plt.yticks(rotation=360, fontsize=30)

plt.show()

### Bivariate Analysis of Categorical-Categorical to Find the Maximum % Clients with Loan-Payment Difficulties

In [ ]:
 pr=app[['AMT_INCOME_TYPE', 'TARGET']].groupby(['AMT_INCOME_TYPE'],as_index=False).mean() 

In [ ]:
pr

In [ ]:
def biplot(data, feature, title):
    
    temp= data[feature].value_counts()
    
    #Calculate the percentage of target=1 for per category value
    
    perc= data[[feature, 'TARGET']].groupby([feature], as_index=False).mean()
    perc.sort_values(by= 'TARGET', ascending=False, inplace= True)
    
    fig= make_subplots(rows= 1, cols= 2,subplot_titles=("Count of "+ title,"% of Loan Payment difficulties within each category"))
    fig.add_trace(go.Bar(x= temp.index, y= temp.values), row= 1, col= 1)
    fig.add_trace(go.Bar(x= perc[feature].to_list(), y= perc['TARGET'].to_list()), row= 1, col= 2)
    
    fig['layout']['xaxis']['title']= feature
    fig['layout']['xaxis2']['title']= feature
    fig['layout']['yaxis']['title']='Count'
    fig['layout']['yaxis2']['title']='% of Loan Payment Difficulties'
    fig.update_layout(height=600, width=1000, title_text=title, showlegend=False)
    
    fig.show()

In [ ]:
biplot(app ,'AMT_INCOME_TYPE','Income range')

In [ ]:
app.select_dtypes(include="category").columns

In [ ]:
for i in app.select_dtypes(include="category").columns:
    biplot(app , i, i)

#### Distribution of CODE_GENDER with respect to AMT_INCOME_RANGE to find maximum % Loan-Payment Difficulties using pivot table

In [ ]:
table= pd.pivot_table(data= app, values= 'TARGET', index= ['CODE_GENDER', 'AMT_INCOME_TYPE'], columns= ['NAME_EDUCATION_TYPE'], aggfunc= np.mean)

In [ ]:
table

### Check Correlations in the data visually. For that make a list of all numeric features.

In [ ]:
num_cols= app.select_dtypes(exclude='category').columns
num_cols

In [ ]:
len(num_cols)

### Pairplot for Target 0 (Loan - No Payment Difficulties)

In [ ]:
pair = target0[['TARGET','AMT_CREDIT', 'AMT_ANNUITY', 'AMT_INCOME_TOTAL', 'AMT_GOODS_PRICE', 'DAYS_BIRTH','CNT_CHILDREN','DAYS_EMPLOYED']].fillna(0)
sns.pairplot(pair)


### Pairplot for Target 1 (Loan - Payment Difficulties)

In [ ]:
pair = target1[['TARGET','AMT_CREDIT', 'AMT_ANNUITY', 'AMT_INCOME_TOTAL', 'AMT_GOODS_PRICE', 'DAYS_BIRTH','CNT_CHILDREN','DAYS_EMPLOYED']].fillna(0)
sns.pairplot(pair)

### Correlation Between Numerical Values Using HeatMap

In [ ]:
# Correlation By seperating data in to two

corr0= app.iloc[0: , 4:]
corr1= app.iloc[0:, 4:]

t0= corr0.corr(method = 'spearman') # t0 - Corelations distibuted according rank wise for target 0
t1= corr1.corr(method = 'spearman') # t1 - Corelations distibuted according rank wise for target 1

In [ ]:
def targets_corr(data,title):
    plt.figure(figsize=(15, 10))
    
    mask= np.zeros_like(data)
    mask[np.triu_indices_from(mask)]=True
    with sns.axes_style("white"):
        ax= sns.heatmap(data, mask=mask,cmap='RdYlGn')           


### For TARGET 0

In [ ]:
targets_corr(data=t0,title='Correlation for Target 0')

In [ ]:
Insight:

    > AMT_CREDIT is inversely proportional to the DAYS_BIRTH , peoples belongs to low-age group taking high Credit amount and vice-versa

    > AMT_CREDIT is inversely proportional to the CNT_CHILDREN, means Credit amount is higher for less children count client have and vice-versa.

    > AMT_INCOME_TOTAL is inversely proportional to the CNT_CHILDREN, means more income for less children client have and vice-versa.

    > less children client have in densely populated area.

    > AMT_CREDIT is higher to densely populated area.

    > AMT_INCOME_TOTAL is also higher in densely populated area.


### For TARGET 1

In [ ]:
targets_corr(data=t1,title='Correlation for Target 1')

In [ ]:
ar